In [ ]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys
import math
import numpy as np
sys.path.append('../../pytools')
import d2l

d2l.gpu_mem_init()

: 

In [ ]:
export_saved_model = True
export_h5_model = True
def nin_block(num_channels, kernel_size, strides, padding):
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(num_channels, kernel_size, strides=strides, padding=padding, activation='relu', name='zjtest_conv2d_L1'),
        tf.keras.layers.Conv2D(num_channels, kernel_size=1, activation='relu',name='zjtest_conv2d_L2'),
        tf.keras.layers.Conv2D(num_channels, kernel_size=1, activation='relu',name='zjtest_conv2d_L3')
    ])

: 

In [ ]:

input_shape=(224,224,1)
input_layer = tf.keras.layers.Input(name='nin_model_input_zj',shape=input_shape)

def net():
    return tf.keras.models.Sequential([
        input_layer,
        nin_block(96, kernel_size=11, strides=4, padding='valid'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        nin_block(256, kernel_size=5, strides=1, padding='same'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        nin_block(384, kernel_size=3, strides=1, padding='same'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Dropout(0.5),
        nin_block(10, kernel_size=3, strides=1, padding='same'),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Reshape((1,1,10)),
        tf.keras.layers.Flatten(name="nin_net_output_zj")
    ])

: 

In [ ]:
X = tf.random.uniform((1, 224, 224, 1))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

: 

In [ ]:
lr, num_epochs, batch_size = 0.1, 1, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
nin_net = d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

: 

In [ ]:
print(nin_net.summary())

: 

In [ ]:
if export_h5_model:
    nin_net.save_weights(filepath='/home/ubuntu/work/data/nin-net/h5/nin_net.weights.h5')

: 

In [ ]:
clone = net()
clone_test = net()
print(clone_test.summary())

: 

In [ ]:
for x,y in train_iter:
    clone(x)
    break
print(clone.summary())

: 

In [ ]:
if export_h5_model:
    clone.load_weights(filepath='/home/ubuntu/work/data/nin-net/h5/nin_net.weights.h5')

: 

In [ ]:
#保存、加载模型，网络模式
nin_net.save("nin_model.keras")

: 

In [ ]:
infer = tf.keras.models.load_model("nin_model.keras")
for x,y in train_iter:
    infer(x)
    break
print(len(infer.layers))

: 

In [ ]:
if export_saved_model:
    nin_net.export("/home/ubuntu/work/data/nin-net/saved_model")

: 

In [ ]:
if export_saved_model:
    info_sv = tf.keras.layers.TFSMLayer('/home/ubuntu/work/data/nin-net/saved_model', call_endpoint='serving_default')
    for x,y in train_iter:
        output = info_sv(x)
        print(output)
        break
    print(info_sv)

: 